# Задача 1 Химический анализ воды

На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad средствами библиотеки Pandas сформируйте поле выводов по каждому анализирумомому параметру. Например, по показателю pH получен результат 8.4 единицы pH при нормативе от 6 до 9 единиц pH. Т.о. по данному показателю результат анализа в норме. Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, 6 >= pH >= 9. В итоговом DataFrame столбец "Показатель" сделайте индексным.

In [124]:
import pandas as pd
import numpy as np
import csv
import requests # for web-download
import io # for web-download
import re # for data processing

In [125]:
url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('UTF8')))https://yandex.ru/maps?ll=65.49689%2C57.162312&mode=routes&rtext=~57.162312%2C65.49689&z=17
### Если не работает загрузка on-line
# df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
display(df)

SyntaxError: invalid syntax (<ipython-input-125-43329fbe8e31>, line 3)

In [126]:
df=pd.read_csv("data-20160608T1215-structure-20160608T1215.csv", sep=',')
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [127]:
df.columns

Index(['Показатель', 'Единица измерений', 'Результат анализа', 'Норматив'], dtype='object')

In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Показатель         14 non-null     object
 1   Единица измерений  14 non-null     object
 2   Результат анализа  14 non-null     object
 3   Норматив           14 non-null     object
dtypes: object(4)
memory usage: 576.0+ bytes


In [129]:
#заменила значение "б\цвета" на 0
df.loc[2,'Результат анализа']=0
df

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,0,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [130]:
#посмотрела типы значений в столбцах
df.dtypes

Показатель           object
Единица измерений    object
Результат анализа    object
Норматив             object
dtype: object

In [131]:
#значениям по столбцу результат анализа присвоила числовой тип 
df = df.astype({'Результат анализа': np.float64})

In [132]:
df.dtypes

Показатель            object
Единица измерений     object
Результат анализа    float64
Норматив              object
dtype: object

In [133]:
#задала условия соответствия значения результатов анализа нормативам.
result = [
    df['Показатель'].eq('pH') & df['Результат анализа'].le(9)& df['Результат анализа'].ne(3),
    df['Показатель'].eq('Запах') & df['Результат анализа'].le(3),
    df['Показатель'].eq('Цветность') & df['Результат анализа'].le(30),
    df['Показатель'].eq('Жёсткость') & df['Результат анализа'].le(10)& df['Результат анализа'].ne(7),
    df['Показатель'].eq('Аммиак и аммоний-ион (по азоту)') & df['Результат анализа'].le(1.5),
    df['Показатель'].eq('Нитриты (по NO2)') & df['Результат анализа'].le(3.3),
    df['Показатель'].eq('Нитраты (по NO3)') & df['Результат анализа'].le(45),
    df['Показатель'].eq('Фосфаты (P)') & df['Результат анализа'].le(3.5),
    df['Показатель'].eq('Хлориды (Cl)') & df['Результат анализа'].le(350),
    df['Показатель'].eq('Сульфаты (SO4)') & df['Результат анализа'].le(500),
    df['Показатель'].eq('Железо (включая хлорное железо) по Fe') & df['Результат анализа'].le(0.3),
    df['Показатель'].eq('Нефть') & df['Результат анализа'].le(0.3),
    df['Показатель'].eq('Общая минерализация (сухой остаток)') & df['Результат анализа'].le(1000),
    df['Показатель'].eq('Окисляемость перманганатная') & df['Результат анализа'].le(0.5),
    
]

choices = ['норма','норма','норма','норма','норма','норма','норма','норма','норма','норма','норма','норма','норма','норма']

#добавила новый столбец с выводом по каждому анализирумомому параметру
df['Вывод'] = np.select(result, choices, default='превышает норму')
df

,Показатель,Единица измерений,Результат анализа,Норматив,Вывод
0,pH,единицы pH,8.400,в пределах 6-9,норма
1,Запах,баллы,1.000,не более 2-3,норма
2,Цветность,градусы,0.000,не более 30,норма
3,Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,норма
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",норма
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",норма
6,Нитраты (по NO3),мг/дм3,24.000,не более 45,норма
7,Фосфаты (P),мг/дм3,0.360,"не более 3,5",норма
8,Хлориды (Cl),мг/дм3,200.000,не более 350,норма
9,Сульфаты (SO4),мг/дм3,189.500,не более 500,норма


In [134]:
#столбец "Показатель" сделала индексным
df = df.set_index('Показатель')
df

,Единица измерений,Результат анализа,Норматив,Вывод
Показатель,,,,
pH,единицы pH,8.400,в пределах 6-9,норма
Запах,баллы,1.000,не более 2-3,норма
Цветность,градусы,0.000,не более 30,норма
Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,норма
Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",норма
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",норма
Нитраты (по NO3),мг/дм3,24.000,не более 45,норма
Фосфаты (P),мг/дм3,0.360,"не более 3,5",норма
Хлориды (Cl),мг/дм3,200.000,не более 350,норма
